In [1]:
# import numpy as np
# import pandas as pd
# from pandas.tseries.offsets import MonthEnd

# # raw_data = pd.read_excel(
# #     "/Users/drest/Downloads/Book2.xlsx",
# #     sheet_name="Country_Avg",
# # )

# # # Unpivot the DataFrame so each year/month is a row, not a column
# # raw_data = raw_data.melt(id_vars=["Unnamed: 0"], var_name="year", value_name="value")
# # raw_data = raw_data.rename(columns={"Unnamed: 0": "month"})

# # raw_data["eom"] = pd.to_datetime(raw_data["year"].astype(str) + "-" + raw_data["month"].astype(str).str.strip() + "-01", format="%Y-%b-%d") + MonthEnd(0)
# # raw_data.drop(columns=["year", "month"], inplace=True)
# # weekly_data = pd.read_parquet(".data/extracted/porkcolombia.parquet.snappy")
# # weekly_data = weekly_data.loc[weekly_data["ts"] >= pd.to_datetime("2023-12-31")]
# # weekly_data["eom"] = weekly_data["ts"] + MonthEnd(0)
# # weekly_data_ = weekly_data.groupby("eom").agg({"average": "mean"}).reset_index().sort_values("eom", ascending=False)
# # combined = raw_data.merge(weekly_data_, on="eom", how="left")
# # combined["price"] = combined[["value", "average"]].mean(axis=1, skipna=True)
# # combined.drop(columns=["value", "average"], inplace=True)
# # combined.sort_values("eom", ascending=False, inplace=True)
# # combined.reset_index(drop=True).to_parquet(".data/extracted/porkcolombia_country_avg.parquet.snappy", index=False)
# country_avg = pd.read_parquet(".data/extracted/porkcolombia_country_avg.parquet.snappy")
# regional_data = pd.read_excel(
#     "/Users/drest/Downloads/Book2.xlsx",
#     sheet_name="Regional",
# )
# regional_data.fillna(method="ffill", inplace=True)
# antioquia = regional_data[["Mes", "Antioquia"]].rename(columns={"Mes": "eom", "Antioquia": "price"})
# antioquia["eom"] = antioquia["eom"] + MonthEnd(0)
# antioquia = antioquia.sort_values("eom", ascending=False).reset_index(drop=True).set_index("eom")
# antioquia.loc[pd.to_datetime("2025-12-31"), "price"] = 8_126
# antioquia.loc[:, "series_id"] = "Antioquia"
# country_avg.loc[:, "series_id"] = "Country_Avg"
# antioquia.reset_index(inplace=True)
# price_data = pd.concat([antioquia, country_avg], ignore_index=True, axis=0).sort_values(["series_id", "eom"], ascending=False).reset_index(drop=True)


# import pandas as pd
# from pandas.tseries.offsets import MonthEnd


# raw_country_production_data = pd.read_excel(
#     "/Users/drest/Downloads/Book2.xlsx",
#     sheet_name="Country_production_avg",
# )

# unpivoted_country_production_data = raw_country_production_data.melt(id_vars=["Mes"], var_name="year", value_name="value").rename(columns={"Mes": "month"})
# unpivoted_country_production_data["eom"] = pd.to_datetime(unpivoted_country_production_data["year"].astype(str) + "-" + unpivoted_country_production_data["month"].astype(str).str.strip() + "-01", format="%Y-%b-%d") + MonthEnd(0)
# unpivoted_country_production_data.drop(columns=["year", "month"], inplace=True)
# unpivoted_country_production_data.rename(columns={"value": "production"}, inplace=True)
# unpivoted_country_production_data.sort_values("eom", ascending=False, inplace=True)
# unpivoted_country_production_data.reset_index(drop=True, inplace=True)
# unpivoted_country_production_data.loc[:, "series_id"] = "Country_Avg"

# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression

# # Prepare data for polynomial regression
# country_production_sorted = unpivoted_country_production_data.sort_values("eom").reset_index(drop=True)
# country_production_sorted["numeric_time"] = (country_production_sorted["eom"] - country_production_sorted["eom"].min()).dt.days
# X_train = country_production_sorted["numeric_time"].values.reshape(-1, 1)
# y_train = country_production_sorted["production"].astype(float).values

# # Remove NaN values, if any
# mask_not_nan = ~pd.isna(y_train)
# X_train = X_train[mask_not_nan]
# y_train = y_train[mask_not_nan]

# poly = PolynomialFeatures(degree=3)
# X_poly = poly.fit_transform(X_train)

# model = LinearRegression()
# model.fit(X_poly, y_train)

# # Predict for 2025-12-31
# target_date = pd.to_datetime("2025-12-31")
# target_numeric_time = (target_date - country_production_sorted["eom"].min()).days
# X_pred = poly.transform([[target_numeric_time]])
# production_2025_12_31 = model.predict(X_pred)[0]

# # Optionally, append to DataFrame (if needed for downstream use)
# predicted_row = {
#     "eom": target_date,
#     "production": production_2025_12_31,
#     "series_id": "Country_Avg"
# }
# unpivoted_country_production_data = pd.concat([
#     unpivoted_country_production_data,
#     pd.DataFrame([predicted_row])
# ], ignore_index=True).dropna()

# raw_regional_production_data = pd.read_excel(
#     "/Users/drest/Downloads/Book2.xlsx",
#     sheet_name="Regional_production_avg",
# )

# regional_production_data = raw_regional_production_data.T
# regional_production_data.columns = regional_production_data.iloc[0]
# regional_production_data.drop(regional_production_data.index[0], inplace=True)
# regional_production_data.reset_index(inplace=True)
# regional_production_data.rename(columns={"index": "month"}, inplace=True)
# regional_production_data["eom"] = pd.to_datetime(regional_production_data["month"].astype(str).str.strip()) + MonthEnd(0)
# regional_production_data.drop(columns=["month"], inplace=True)
# regional_production_data = regional_production_data[["eom", "Antioquia"]].rename(columns={"Antioquia": "production"})
# regional_production_data.loc[:, "series_id"] = "Antioquia"

# regional_production_data.columns.name = None
# regional_production_data.index.name = None
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression

# # Prepare data for autoregression (polynomial)
# regional_production_data_sorted = regional_production_data.sort_values("eom").reset_index(drop=True)

# # Use only numeric index as time to predict next value at t+1 (2025-12-31)
# regional_production_data_sorted['numeric_time'] = (regional_production_data_sorted['eom'] - regional_production_data_sorted['eom'].min()).dt.days

# # Use all but last for X_train, and corresponding for y_train
# X_train = regional_production_data_sorted['numeric_time'].values.reshape(-1, 1)
# y_train = regional_production_data_sorted['production'].astype(float).values

# # Remove missing (NaN) production data, just in case
# mask_not_nan = ~pd.isna(y_train)
# X_train = X_train[mask_not_nan]
# y_train = y_train[mask_not_nan]

# # Polynomial features (degree can be changed as needed)
# poly = PolynomialFeatures(degree=3)
# X_poly = poly.fit_transform(X_train)

# model = LinearRegression()
# model.fit(X_poly, y_train)

# # Predict for 2025-12-31
# target_date = pd.to_datetime("2025-12-31")
# target_numeric_time = (target_date - regional_production_data_sorted['eom'].min()).days
# X_pred = poly.transform([[target_numeric_time]])
# predicted_production = model.predict(X_pred)[0]

# # Insert the prediction for 2025-12-31 if it's not there
# if not (regional_production_data_sorted['eom'] == target_date).any():
#     # Insert at the end
#     new_row = pd.DataFrame({
#         'eom': [target_date],
#         'production': [predicted_production],
#         'series_id': ["Antioquia"]
#     })
#     # Ensure columns match (add missing ones if necessary)
#     for col in regional_production_data_sorted.columns:
#         if col not in new_row.columns:
#             new_row[col] = [None]
#     regional_production_data = pd.concat([regional_production_data, new_row[regional_production_data.columns]], ignore_index=True)


# regional_production_data.sort_values("eom", ascending=False, inplace=True)
# regional_production_data.reset_index(drop=True, inplace=True, names=None)
# regional_production_data
# production_data = pd.concat([unpivoted_country_production_data, regional_production_data], ignore_index=True, axis=0).sort_values(["series_id", "eom"], ascending=False).reset_index(drop=True)
# production_data

# combined = pd.merge(price_data, production_data, on=["eom", "series_id"], how="inner")

# raw_consumption_data = pd.read_excel(
#     "/Users/drest/Downloads/Book2.xlsx",
#     sheet_name="Consumption",
# )

# raw_consumption_data
# from sklearn.linear_model import LinearRegression
# import numpy as np

# # Prepare the data for regression
# consumption_df = raw_consumption_data.copy()
# X = consumption_df[['year']].values
# y = consumption_df['per_capita_consumption'].values

# # Fit the linear regression model
# model = LinearRegression()
# model.fit(X, y)

# # Predict per capita consumption for 2025
# year_2025 = np.array([[2025]])
# per_capita_2025 = model.predict(year_2025)[0]

# print(f"Predicted per_capita_consumption for 2025: {per_capita_2025:.2f}")

# raw_consumption_data.loc[len(raw_consumption_data.index)] = pd.Series({"year": 2025, "per_capita_consumption": per_capita_2025})
# raw_consumption_data.sort_values("year", ascending=True, inplace=True)
# raw_consumption_data.reset_index(drop=True, inplace=True)

# import pandas as pd
# import numpy as np
# from pandas.tseries.offsets import MonthEnd

# # Create a monthly date range from the first year to the last year
# monthly_dates = pd.date_range(
#     start=f"{raw_consumption_data['year'].min().astype(int)}-01-01", 
#     end=f"{raw_consumption_data['year'].max().astype(int)}-12-31", 
#     freq='M'
# )
# monthly_consumption_data = pd.DataFrame({
#     'eom': monthly_dates + MonthEnd(0),
#     'year': monthly_dates.year,
# })
# raw_monthly_consumption_data = pd.merge(monthly_consumption_data, raw_consumption_data, on="year", how="left").drop(columns=["year"]).sort_values("eom", ascending=False).reset_index(drop=True)

# regional_monthly_consumption_data = raw_monthly_consumption_data.copy()
# regional_monthly_consumption_data.loc[:, "series_id"] = "Antioquia"
# country_monthly_consumption_data = raw_monthly_consumption_data.copy()
# country_monthly_consumption_data.loc[:, "series_id"] = "Country_Avg"

# consumption_data = pd.concat([regional_monthly_consumption_data, country_monthly_consumption_data], ignore_index=True, axis=0).sort_values(["series_id", "eom"], ascending=False).reset_index(drop=True)
# consumption_data

# combined = pd.merge(combined, consumption_data, on=["eom", "series_id"], how="left")
# combined = combined.loc[combined["eom"] >= pd.to_datetime("2020-01-01")]
# combined.to_parquet(".data/extracted/porkcolombia_combined.parquet.snappy", index=False)
# combined

In [2]:
import pandas as pd
import numpy as np

combined = pd.read_parquet(".data/extracted/porkcolombia_combined.parquet.snappy")
combined["year"] = combined["eom"].dt.year
combined["month"] = combined["eom"].dt.month
combined["time_idx"] = combined.groupby("series_id").cumcount(ascending=False)
combined["price"] = combined["price"].astype(np.float64)
combined["production"] = combined["production"].astype(np.float64)
combined["per_capita_consumption"] = combined["per_capita_consumption"].astype(np.float64)
combined["per_capita_consumption"] = combined["per_capita_consumption"] * combined["price"]
combined.sort_values(["eom"], ascending=True, inplace=True)
combined.reset_index(drop=True, inplace=True)
combined = combined.loc[combined["eom"] >= pd.to_datetime("2020-01-01")]


In [3]:
# from ts_piggies.forecast.tft import TFTConfig, TFTForecastWrapper
# from ts_piggies.helpers.scaler import MinMaxScalerWrapper

# minmax_scaler = MinMaxScalerWrapper()
# combined_scaled = minmax_scaler.fit_transform(combined, ["price", "production", "per_capita_consumption"])

# wrapper = TFTForecastWrapper(
#     combined_scaled,
#     TFTConfig(
#         learning_rate=0.003,
#         hidden_size=64,
#         attention_head_size=8,
#         dropout=0.01,
#         hidden_continuous_size=64,
#         batch_size=32,
#         max_epochs=100,
#         gradient_clip_val=0.01,
#         reduce_on_plateau_patience=10,
#     ),
#     max_encoder_length=64,
#     max_prediction_length=7,
# )

# result = wrapper.fit_predict(simulations=True)

# results = pd.DataFrame({"eom": result.dates, "price": result.mean})

# results = minmax_scaler.inverse_transform(results, ["price"])

# correction = combined.iloc[-1]["price"] - results.iloc[0]["price"]
# results["price"] = results["price"].astype(np.float64) + correction

# results

In [ ]:
import logging
from ts_piggies.helpers.scaler import MinMaxScalerWrapper
from ts_piggies.forecast.grid import ErrorMetric, GridSearch, GridSearchConfig
from ts_piggies.forecast.tft import TFTConfig, TFTForecastWrapper

logger = logging.getLogger("ts_piggies")
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

max_encoder_length = 64
max_prediction_length = 12

minmax_scaler = MinMaxScalerWrapper()
combined_scaled = minmax_scaler.fit_transform(combined, ["price", "production", "per_capita_consumption"])

custom_grid = {
    "learning_rate": [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04],  # 7 values - expanded range
    "hidden_size": [128, 256, 512],  # 4 values - includes smaller and larger models
    "attention_head_size": [4, 6, 8],  # 4 values - varied attention mechanisms
    "dropout": [0.1, 0.15, 0.2, 0.25],  # 5 values - regularization range
    "hidden_continuous_size": [16, 32, 48, 64],  # 4 values - continuous feature processing
    "batch_size": [32, 64, 128],  # 3 values - different batch sizes
    "max_epochs": [100, 120, 150, 250, 300],  # 5 values - training duration variation
    "gradient_clip_val": [0.05, 0.1, 0.15, 0.2],  # 4 values - gradient clipping
    "reduce_on_plateau_patience": [4, 6, 8],  # 3 values - learning rate scheduling
}

grid_config = GridSearchConfig(
    config_type=TFTConfig,
    model_type=TFTForecastWrapper,
    hyperparameter_grid=custom_grid,
    segment_columns=["series_id"],
    sort_columns=["eom"],
    test_column="price",
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    n_simulations=1000,
    error_metric=ErrorMetric.MAPE,
    validation_length=0.1,
)

# Initialize grid search
grid_search = GridSearch(
    data=combined_scaled,
    config=grid_config,
)

# Run grid search with at least 40 models for statistically convincing results
# Using random_sample=True ensures diverse hyperparameter combinations
# This provides better coverage of the hyperparameter space
forecast_results = grid_search.search(max_models=250, random_sample=True).fit_predict()

In [ ]:

# Inverse transform the probabilistic forecasts back to original scale

# Create a DataFrame with the forecast values for inverse transformation
from ts_piggies.forecast.abstract import ForecastResult, ProbabilisticForecastResult


forecast_df = pd.DataFrame({
    "price": forecast_results.mean
})

# Inverse transform the mean forecast
forecast_df_inverse = minmax_scaler.inverse_transform(forecast_df, columns=["price"])
mean_inverse = forecast_df_inverse["price"].tolist()

forecast_df_inverse = pd.DataFrame({
    "price": forecast_results.median
})

# Inverse transform the median forecast
forecast_df_inverse = minmax_scaler.inverse_transform(forecast_df, columns=["price"])
median_inverse = forecast_df_inverse["price"].tolist()


nearer_to_zero_mean = (combined.iloc[-1]["price"] - mean_inverse[0])
nearer_to_zero_median = (combined.iloc[-1]["price"] - median_inverse[0])

correction = nearer_to_zero_mean if abs(nearer_to_zero_mean) < abs(nearer_to_zero_median) else nearer_to_zero_median

mean_inverse = [x + correction for x in mean_inverse]
median_inverse = [x + correction for x in median_inverse]

# Inverse transform the std (standard deviation scales with the same transformation)
# Note: std should be scaled by the same factor as the mean
std_df = pd.DataFrame({
    "price": forecast_results.std
})
std_inverse = minmax_scaler.inverse_transform(std_df, columns=["price"]).values.flatten().tolist()
# std_inverse = forecast_df_inverse["price"].values * np.array(forecast_results.std)

# Inverse transform all quantiles
quantiles_inverse = {}
for quantile_name, quantile_values in forecast_results.quantiles.items():
    quantile_df = pd.DataFrame({
        "price": quantile_values
    })
    quantile_df_inverse = minmax_scaler.inverse_transform(quantile_df, columns=["price"])
    quantiles_inverse[quantile_name] = (quantile_df_inverse["price"] + correction).tolist()

# Inverse transform individual forecasts
individual_forecasts_inverse = []
for forecast_result in forecast_results.individual_forecasts:
    individual_df = pd.DataFrame({
        "price": forecast_result.forecast
    })
    individual_df_inverse = minmax_scaler.inverse_transform(individual_df, columns=["price"])
    
    # Create new ForecastResult with inverse transformed values
    forecast_result_inverse = ForecastResult(
        model_name=forecast_result.model_name,
        config=forecast_result.config,
        forecast=(individual_df_inverse["price"] + correction).tolist(),
        dates=forecast_result.dates,
        train_loss=forecast_result.train_loss,
    )
    individual_forecasts_inverse.append(forecast_result_inverse)


# Create new ProbabilisticForecastResult with inverse transformed values
probabilistic_forecast_inverse = ProbabilisticForecastResult(
    dates=forecast_results.dates,
    mean=mean_inverse,
    median=median_inverse,
    std=std_inverse,
    quantiles=quantiles_inverse,
    individual_forecasts=individual_forecasts_inverse,
    n_models=forecast_results.n_models,
)

# Display inverse transformed results
print(f"\n{'='*60}")
print(f"Inverse Transformed Probabilistic Forecast Summary")
print(f"{'='*60}")
print(f"Number of models: {probabilistic_forecast_inverse.n_models}")
print(f"Forecast period: {len(probabilistic_forecast_inverse.mean)} months")
print(f"\nMean forecast (first 3 months) - Original Scale:")
for i, (date, mean_val) in enumerate(zip(probabilistic_forecast_inverse.dates[:3], probabilistic_forecast_inverse.mean[:3])):
    std_val = probabilistic_forecast_inverse.std[i]
    print(f"  {date}: {mean_val:.2f} ± {std_val:.2f}")

# Plot inverse transformed forecast with historical data (original scale)
probabilistic_forecast_inverse


In [ ]:
forecast = pd.DataFrame(
    data={
        "eom": probabilistic_forecast_inverse.dates,
        "mean": probabilistic_forecast_inverse.mean,
        "median": probabilistic_forecast_inverse.median,
        "std": probabilistic_forecast_inverse.std,
        **{ f"quantile_{int(float(col) * 100)}": li for col, li in probabilistic_forecast_inverse.quantiles.items()},
    },
)

forecast



In [ ]:
# Histogram comparing forecast distribution vs historical price distribution

import matplotlib.pyplot as plt
import numpy as np

# Collect all forecast values from all individual models (already inverse transformed)
all_forecast_values = []
for forecast_result in probabilistic_forecast_inverse.individual_forecasts:
    all_forecast_values.extend(forecast_result.forecast)

# Get historical price data (original scale, not scaled)
historical_prices = combined["price"].dropna().values

# Create figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Overlaid histograms
ax1 = axes[0]
ax1.hist(
    historical_prices, 
    bins=30, 
    alpha=0.6, 
    label=f'Historical Prices (n={len(historical_prices)})', 
    color='steelblue',
    density=True,
    edgecolor='black',
    linewidth=1.2
)
ax1.hist(
    all_forecast_values, 
    bins=30, 
    alpha=0.6, 
    label=f'All Forecasts (n={len(all_forecast_values)})', 
    color='coral',
    density=True,
    edgecolor='black',
    linewidth=1.2
)

# Add vertical lines for means
hist_mean = np.mean(historical_prices)
forecast_mean = np.mean(all_forecast_values)
ax1.axvline(hist_mean, color='steelblue', linestyle='--', linewidth=2, label=f'Historical Mean: {hist_mean:.2f}')
ax1.axvline(forecast_mean, color='coral', linestyle='--', linewidth=2, label=f'Forecast Mean: {forecast_mean:.2f}')

ax1.set_xlabel('Price (Original Scale)', fontsize=12)
ax1.set_ylabel('Density', fontsize=12)
ax1.set_title('Distribution Comparison: Historical vs Forecasts', fontsize=14, fontweight='bold')
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Right plot: Side-by-side box plots
ax2 = axes[1]
box_data = [historical_prices, all_forecast_values]
box_labels = [f'Historical\n(n={len(historical_prices)})', f'All Forecasts\n(n={len(all_forecast_values)})']
bp = ax2.boxplot(box_data, labels=box_labels, patch_artist=True, widths=0.6)

# Color the boxes
bp['boxes'][0].set_facecolor('steelblue')
bp['boxes'][0].set_alpha(0.7)
bp['boxes'][1].set_facecolor('coral')
bp['boxes'][1].set_alpha(0.7)

# Color the medians
for median in bp['medians']:
    median.set_color('black')
    median.set_linewidth(2)

ax2.set_ylabel('Price (Original Scale)', fontsize=12)
ax2.set_title('Box Plot Comparison: Historical vs Forecasts', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Add statistics text
stats_text = f"""
Statistics Comparison:

Historical Prices:
  Mean: {np.mean(historical_prices):.2f}
  Median: {np.median(historical_prices):.2f}
  Std: {np.std(historical_prices):.2f}
  Min: {np.min(historical_prices):.2f}
  Max: {np.max(historical_prices):.2f}

All Forecasts:
  Mean: {np.mean(all_forecast_values):.2f}
  Median: {np.median(all_forecast_values):.2f}
  Std: {np.std(all_forecast_values):.2f}
  Min: {np.min(all_forecast_values):.2f}
  Max: {np.max(all_forecast_values):.2f}
"""

print(stats_text)

plt.tight_layout()
plt.show()

# Additional: Create a single combined histogram with better styling
fig2, ax3 = plt.subplots(figsize=(12, 6))

# Create histogram with better binning
n_bins = max(30, int(np.sqrt(len(historical_prices)) + np.sqrt(len(all_forecast_values))))
bins = np.linspace(
    min(np.min(historical_prices), np.min(all_forecast_values)),
    max(np.max(historical_prices), np.max(all_forecast_values)),
    n_bins
)

ax3.hist(
    historical_prices, 
    bins=bins, 
    alpha=0.5, 
    label=f'Historical Prices (n={len(historical_prices)})', 
    color='steelblue',
    density=True,
    edgecolor='darkblue',
    linewidth=1.5
)
ax3.hist(
    all_forecast_values, 
    bins=bins, 
    alpha=0.5, 
    label=f'All Forecasts (n={len(all_forecast_values)})', 
    color='coral',
    density=True,
    edgecolor='darkred',
    linewidth=1.5
)

# Add mean lines
ax3.axvline(hist_mean, color='steelblue', linestyle='--', linewidth=2.5, 
            label=f'Historical Mean: {hist_mean:.2f}', alpha=0.8)
ax3.axvline(forecast_mean, color='coral', linestyle='--', linewidth=2.5, 
            label=f'Forecast Mean: {forecast_mean:.2f}', alpha=0.8)

# Add median lines
hist_median = np.median(historical_prices)
forecast_median = np.median(all_forecast_values)
ax3.axvline(hist_median, color='steelblue', linestyle=':', linewidth=2, 
            label=f'Historical Median: {hist_median:.2f}', alpha=0.6)
ax3.axvline(forecast_median, color='coral', linestyle=':', linewidth=2, 
            label=f'Forecast Median: {forecast_median:.2f}', alpha=0.6)

ax3.set_xlabel('Price (Original Scale)', fontsize=13, fontweight='bold')
ax3.set_ylabel('Density', fontsize=13, fontweight='bold')
ax3.set_title('Price Distribution: Historical Data vs Ensemble Forecasts', fontsize=15, fontweight='bold')
ax3.legend(loc='best', fontsize=11, framealpha=0.9)
ax3.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()


In [ ]:
# Comprehensive line plot with historical data, forecast quantiles, mean, median, std, and trendline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from pandas.tseries.offsets import MonthEnd

# Set seaborn style
sns.set_style("whitegrid")
sns.set_palette("husl")

# Prepare historical data
historical_df = combined[["eom", "price", "series_id"]].copy()
historical_df["eom"] = pd.to_datetime(historical_df["eom"])

std_multiplier = .5
# Prepare forecast data
forecast_dates = pd.to_datetime(probabilistic_forecast_inverse.dates)
forecast_df = pd.DataFrame({
    "eom": forecast_dates,
    "mean": probabilistic_forecast_inverse.mean,
    "median": probabilistic_forecast_inverse.median,
    "std": probabilistic_forecast_inverse.std,
    "quantile_10": probabilistic_forecast_inverse.quantiles.get("0.1", []),
    "quantile_25": probabilistic_forecast_inverse.quantiles.get("0.25", []),
    "quantile_50": probabilistic_forecast_inverse.quantiles.get("0.5", []),
    "quantile_75": probabilistic_forecast_inverse.quantiles.get("0.75", []),
    "quantile_90": probabilistic_forecast_inverse.quantiles.get("0.9", []),
})

# Calculate upper and lower bounds for std
forecast_df["mean_plus_std"] = forecast_df["mean"] + (forecast_df["std"] * std_multiplier)
forecast_df["mean_minus_std"] = forecast_df["mean"] - (forecast_df["std"] * std_multiplier)

# Get unique series
unique_series = historical_df["series_id"].unique()
n_series = len(unique_series)

# Create subplots - one for each series
if n_series > 1:
    fig, axes = plt.subplots(n_series, 1, figsize=(16, 6 * n_series), sharex=True)
    # Ensure axes is always a list/array we can index
    if not isinstance(axes, np.ndarray):
        axes = np.array([axes])
    axes = axes.flatten()
else:
    fig, ax = plt.subplots(figsize=(16, 8))
    axes = [ax]

for idx, series_id in enumerate(unique_series):
    ax = axes[idx]
    
    # Filter historical data for this series
    series_historical = historical_df[historical_df["series_id"] == series_id].copy()
    series_historical = series_historical.sort_values("eom")
    last_historical_date = series_historical["eom"].max()

    # Calculate trendline using only forecast mean
    # Convert forecast dates to numeric for regression
    forecast_date_numeric = (forecast_df["eom"] - forecast_df["eom"].min()).dt.days.values
    trend_slope, trend_intercept, r_value, p_value, std_err = stats.linregress(
        forecast_date_numeric, forecast_df["mean"]
    )
    trendline = trend_slope * forecast_date_numeric + trend_intercept
    
    # Plot historical data
    sns.lineplot(
        data=series_historical,
        x="eom",
        y="price",
        ax=ax,
        label="Historical Price",
        color="black",
        linewidth=2,
        marker="o",
        markersize=4,
        alpha=0.8
    )
    
    # Connect historical price to first forecast mean value
    last_historical_date = series_historical["eom"].max()
    last_historical_price = series_historical["price"].iloc[-1]
    first_forecast_date = forecast_df["eom"].iloc[0]
    first_forecast_mean = forecast_df["mean"].iloc[0]
    
    # Draw connecting line
    ax.plot(
        [last_historical_date, first_forecast_date],
        [last_historical_price, first_forecast_mean],
        color="black",
        linewidth=2,
        alpha=0.8,
        linestyle="-"
    )
    
    # Plot forecast quantile regions (shaded areas)
    # 10th-90th percentile region
    ax.fill_between(
        forecast_df["eom"],
        forecast_df["quantile_10"],
        forecast_df["quantile_90"],
        alpha=0.2,
        color="blue",
        label="10th-90th Percentile"
    )
    
    # 25th-75th percentile region
    ax.fill_between(
        forecast_df["eom"],
        forecast_df["quantile_25"],
        forecast_df["quantile_75"],
        alpha=0.3,
        color="blue",
        label="25th-75th Percentile"
    )
        
    ax.fill_between(
        forecast_df["eom"],
        forecast_df["mean_minus_std"],
        forecast_df["mean_plus_std"],
        alpha=0.15,
        color="orange",
        label=f"Mean ± 1 Std"
    )
    
    # Plot quantile lines
    sns.lineplot(
        x=forecast_df["eom"],
        y=forecast_df["quantile_10"],
        ax=ax,
        color="blue",
        linestyle="--",
        linewidth=1,
        alpha=0.6,
        label="10th Percentile"
    )
    sns.lineplot(
        x=forecast_df["eom"],
        y=forecast_df["quantile_90"],
        ax=ax,
        color="blue",
        linestyle="--",
        linewidth=1,
        alpha=0.6,
        label="90th Percentile"
    )
    
    # Plot mean
    sns.lineplot(
        x=forecast_df["eom"],
        y=forecast_df["mean"],
        ax=ax,
        color="red",
        linewidth=2.5,
        linestyle="-",
        label="Forecast Mean"
    )
    
    # Plot median
    sns.lineplot(
        x=forecast_df["eom"],
        y=forecast_df["median"],
        ax=ax,
        color="green",
        linewidth=2.5,
        linestyle="-",
        label="Forecast Median"
    )
    
    # Plot trendline (only for forecast period)
    sns.lineplot(
        x=forecast_df["eom"],
        y=trendline,
        ax=ax,
        color="purple",
        linewidth=2,
        linestyle=":",
        label=f"Trendline (R²={r_value**2:.3f})"
    )
    
    # Add vertical line to separate historical and forecast
    ax.axvline(
        x=last_historical_date,
        color="gray",
        linestyle="--",
        linewidth=2,
        alpha=0.7,
        label="Forecast Start"
    )
    
    # Formatting
    ax.set_xlabel("Date (End of Month)", fontsize=12, fontweight="bold")
    ax.set_ylabel("Price", fontsize=12, fontweight="bold")
    ax.set_title(
        f"Price Forecast with Uncertainty Intervals - {series_id}\n"
        f"Mean: {forecast_df['mean'].mean():.2f} ± {forecast_df['std'].mean():.2f}, "
        f"Trend: {trend_slope*30:.2f}/month",
        fontsize=14,
        fontweight="bold"
    )
    ax.legend(loc="best", fontsize=10, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle="--")
    
    # Format y-axis to show currency
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*80)
print("Forecast Summary Statistics")
print("="*80)
print(f"Number of series: {n_series}")
print(f"Forecast period: {len(forecast_df)} months")
print(f"\nForecast Statistics (across all dates):")
print(f"  Mean: {forecast_df['mean'].mean():.2f} ± {forecast_df['std'].mean():.2f}")
print(f"  Median: {forecast_df['median'].mean():.2f}")
print(f"  Min (10th percentile): {forecast_df['quantile_10'].min():.2f}")
print(f"  Max (90th percentile): {forecast_df['quantile_90'].max():.2f}")
print(f"  Trend slope: {trend_slope*30:.2f} per month")
print(f"  Trend R²: {r_value**2:.4f}")
print("="*80)
